In [2]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [3]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

# import colab secrets to store login credentials
from google.colab import userdata

# datetime is necessary for caturday check
import datetime
import zoneinfo

In [4]:
# vision transformer documentation - https://huggingface.co/docs/transformers/v4.13.0/en/model_doc/vit
# guide to using vision transformer from DataCamp - https://www.youtube.com/watch?v=WQMDZniQgj4

# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [13]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):

  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  # print('    passed cat test:', would_pass)
  return would_pass

In [14]:
# on colab you can set these userdata properties by clicking the key on the left bar, creating a secret, and giving the colab "notebook access"
BSKY_USERNAME = userdata.get('bsky_username')
BSKY_PASSWORD = userdata.get('bsky_password')

client = Client()
client.login(BSKY_USERNAME, BSKY_PASSWORD)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Self-taught professional software developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2100%, ex streamer, ex melee player. \nGrilling, coding, magic, and posting pictures of my cat. \nNO POLITICS\nhe/him', display_name='Tyro, ft Ricky', followers_count=716, follows_count=1936, indexed_at='2025-01-19T01:42:29.013Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(cid='bafyreihszjgxkh5fmu5je5eldv

In [15]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ Successfully liked post and followed user: {post.author.handle}')
  # TODO - add this friend and the followed-user uri to the database

In [16]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'
EMBEDDED_VID = 'app.bsky.embed.video#view'
FEED_CATURDAY = 'at://did:plc:pmyqirafcp3jqdhrl7crpq7t/app.bsky.feed.generator/aaad4sb7tyvjw'
FEED_SIAMESE = 'at://did:plc:jv3qdc5vxujp6taaa7nte35i/app.bsky.feed.generator/aaac6wmikqyhq'
FEED_CATPICS = 'at://did:plc:q6gjnaw2blty4crticxkmujt/app.bsky.feed.generator/cv:cat'
FEED_CATS = 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats'
FEED_NAME = {FEED_CATURDAY: "'Caturday'", FEED_SIAMESE: "'Siamese Cats'", FEED_CATPICS: "'Cat Pics'", FEED_CATS: "'Cats!'"}
URL_BEGIN = 'https://bsky.app/profile/'
URL_POST = '/post/'

def createPostUrl(feed_post):
  url_handle = feed_post.post.author.handle
  url_ending_index = feed_post.post.uri.find('.feed.post/') + 11
  url_ending = feed_post.post.uri[url_ending_index : ]
  return URL_BEGIN + url_handle + URL_POST + url_ending

# TODO - The way I have this written the function probably breaks if we reach the end of the feed, so I need to write some kind of case to handle that. The feeds I'm using there shouldn't be an end to reach.
# But technically if you set the number of posts to check to like a billion and number of new followers to follow to like a billion it could reach the end and break.
# so please use responsibly

def follow_more_users(post_count, follows_count, feed):
  posts_to_check = post_count
  next_page = ''
  new_follow_count = 0
  page_count = 0
  while posts_to_check > 0:
    print(f'[checking page {page_count} of feed {FEED_NAME[feed]}, {posts_to_check} posts left to check, and have found {new_follow_count} new users to follow]')
    page_count += 1
    limit = min(posts_to_check, 100)
    posts_to_check -= limit
    data = client.app.bsky.feed.get_feed({
        'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
        'limit': limit,
        'cursor': next_page
    }, headers={})
    next_page = data.cursor

    for i, f in enumerate(data.feed):
      you_follow_them = f.post.author.viewer.following
      you_are_followed_by = f.post.author.viewer.followed_by

      if False: # TODO IF POST ALREADY SEEN
        print(f'{i} - skipped. Post {""} has already been viewed.') # add post id here idk
        continue
      else:
        # TODO - add post did to set to mark as seen
        if you_follow_them and you_are_followed_by:
          print(f'{i} 💕 user: {f.post.author.handle} is a mutual follower. Liking this post. {createPostUrl(f)}')
          liked_post = client.like(uri=f.post.uri, cid=f.post.cid).uri
        elif you_follow_them or you_are_followed_by:
          print(f'{i} ✗ user: {f.post.author.handle} {"already follows you." if you_are_followed_by else ""}{"is already being followed." if you_follow_them else ""}')
        ## TODO - add elif check here for if they are in the database as someone who was once our friend? not necessary, could just immediately cycle them back into the friend rotation, but do this if you don't want to readd people
        elif not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
          if f.post.embed.py_type == EMBEDDED_VID:
            print(f'{i} ✗ 🎥 video post: {createPostUrl(f)}')
          else:
            print(f'{i} ✗ no pic for post {i}')
        else:
          print(i, '✓', f.post.embed.images[0].fullsize)
          print(f'    post: {createPostUrl(f)}')
          is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
          if is_cat:
            print(f'    ✓✓ successfully found cat pic at post {i}.')
            like_post_and_add_user(f.post)
            new_follow_count += 1
            if new_follow_count == follows_count:
              print(f'Successfully followed {new_follow_count} new users!')
              return True
          else:
            print(f'    ✓✗ post {i} was not a cat pic')

  print(f'ran out of posts to check for new followers. followed {new_follow_count} new users.')
  return False

    # print(data.feed[0].post.embed.images[0].fullsize)

In [17]:
CATURDAY_DOW = 'Saturday'
USER_TIMEZONE = "US/Eastern" # you should fill this in with your own timezone here
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))

dow = cur_timestamp.strftime("%A")
is_caturday = dow == CATURDAY_DOW

if is_caturday:
  print("IT'S CATURDAY! Checking the Caturday feed for new followers.")
  follow_more_users(500, 100, FEED_CATURDAY)
else:
  print("Sorry, today's not Caturday.")

Sorry, today's not Caturday.


In [18]:
follow_more_users(300, 100, FEED_SIAMESE)

[checking page 0 of feed 'Siamese Cats', 300 posts left to check, and have found 0 new users to follow]
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:2pk3k2exgtkcqsgaujrsyb7d/bafkreiemkfosmdynrlyrqlilldwhcq7hqntbtfjzeh45wejkvizq4deduu@jpeg
    post: https://bsky.app/profile/cryptidqueer.bsky.social/post/3lg4mjc2zkc2c


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'return_tensor'
  return self.preprocess(images, **kwargs)


    found cat label: 12
    found bad label: 8 [917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    ✓✗ post 0 was not a cat pic
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:yzaggurknkrhbywmq5k3flzb/bafkreidtjsw7lyb66ofummwnrdkw6aal527blaytt5obafkbo4pwfnpea4@jpeg
    post: https://bsky.app/profile/rowellreads.bsky.social/post/3lg4miuzryc2s
    found cat label: 2
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 1.
      ✓✓✓ Successfully liked post and followed user: rowellreads.bsky.social
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:73qz62uhmtov2tvyopi57iae/bafkreihpwk757zoipsh3s7gab2wed2vnbnajg7aywtidepojx65oxwaceq@jpeg
    post: https://bsky.app/profile/dustyfyre.bsky.social/post/3lg4minyyms2g
    found cat label: -1
    found bad label: -1 []
    ✓✗ post 2 was not a cat pic
3 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:v3h2jc7ryse5wuhflnirbv3p/bafkreicrq5eyoipnx62ueh6dxpzf7fiyrr5joonccf56toggzgwjt5cu

True

In [19]:
follow_more_users(500, 100, FEED_CATS)

[checking page 0 of feed 'Cats!', 500 posts left to check, and have found 0 new users to follow]
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:55fxth6bgr4qt5e53i3o73dd/bafkreigmfe3j56tqsjw3zummandhgrrgvjhwj5soog2k4ofuumaaxpedja@jpeg
    post: https://bsky.app/profile/mjjaxn.bsky.social/post/3lg4msme7tk2x
    found cat label: 0
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 0.
      ✓✓✓ Successfully liked post and followed user: mjjaxn.bsky.social
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:zy4zfsmppdwuosfi4lxvon32/bafkreigy4mdvv46xfv7lthzjjmx3t3cdwpk3sna2mlijgghhv36xxobvma@jpeg
    post: https://bsky.app/profile/thesinginglettera.bsky.social/post/3lg4mskwxls2p
    found cat label: -1
    found bad label: 0 [916, 'web site, website, internet site, site', 917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    ✓✗ post 1 was not a cat pic
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:5tzv3qbib5amnixyphtzvzw

True

In [20]:
follow_more_users(200, 50, FEED_CATPICS)

[checking page 0 of feed 'Cat Pics', 200 posts left to check, and have found 0 new users to follow]
0 ✗ user: acataday.bsky.social is already being followed.
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:rauk45lsr7avjjngipnsk3iz/bafkreigc3iqzbjmro4zr6eih2zmtsvsa6xpzvq5xxxaggjrdiiepw3ifhy@jpeg
    post: https://bsky.app/profile/supernova8888.bsky.social/post/3lg4n3gwies2s
    found cat label: 0
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 1.
      ✓✓✓ Successfully liked post and followed user: supernova8888.bsky.social
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:yuekj5lbdvrc4kfaqhpqi2qh/bafkreigkim3lzecwcrvgkfiui6i44udjctfy4cxx222njah5h7lz6423zu@jpeg
    post: https://bsky.app/profile/questionxsleep.bsky.social/post/3lg4n3cd6j22g
    found cat label: 0
    found bad label: -1 []
    ✓✓ successfully found cat pic at post 2.
      ✓✓✓ Successfully liked post and followed user: questionxsleep.bsky.social
3 ✓ https://cdn.bsky.app/img/feed_full

False